In [2]:
import logging

logger = logging.getLogger(__name__)

In [4]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from datetime import datetime, timedelta

from short_tracker.data import (
    query_all_sec_metadata, query_uk_si_disclosures, query_mkt_data, query_quotes,
    SHORT_URL_UK, DATE_COL, FUND_COL, ISIN_COL, SHORT_POS_COL,
    SHARE_ISSUER_COL, UK_DISCL_THRESHOLD, FCA_DATE_COL,
)
from short_tracker.processing import (
    check_cur_hist_discl_overlap, remove_dupl_shorts, ffill_discl_data, calc_fund_short_flow_bounds,
    extract_sec_tickers, process_mkt_data, subset_top_shorts,
)

NUM_SUBSET = 20
UK_MKT_TICKER = "VUKE"
ALPHAVANGATE_KEY = ""

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
discl_data, rept_date = query_uk_si_disclosures(SHORT_URL_UK) # exp_upd_time
rept_date

datetime.date(2022, 12, 30)

In [5]:
cur_discl = discl_data['current']

exp_max_discl_date = rept_date - timedelta(days=1)
max_discl_date = cur_discl[FCA_DATE_COL].max().date()

assert max_discl_date <= exp_max_discl_date
max_discl_date

datetime.date(2022, 12, 29)

In [6]:
# subset on current top N total overall shorts + top N individual shorts
top_sec_shorts, top_fund_shorts = subset_top_shorts(cur_discl, NUM_SUBSET)

isins = list({*top_fund_shorts[ISIN_COL].values, *top_sec_shorts[ISIN_COL].values})
print(len(isins))
isins[:5]

30


['GB0032089863',
 'GB00B132NW22',
 'GB0030927254',
 'GB00B4Y7R145',
 'GB0033195214']

In [7]:
sec_metadata, err_isins = query_all_sec_metadata(isins, {"idType": "ID_ISIN", "exchCode": "LN"})
len(err_isins)

0

In [8]:
isin_ticker_map = extract_sec_tickers(sec_metadata)
tickers = [UK_MKT_TICKER, *isin_ticker_map.values()]
query_tickers = [k.rstrip("/") + '.L' for k in tickers]

ticker_map = dict(zip(tickers, query_tickers))

tickers[:5]

['VUKE', 'NXT', 'ASHM', 'ASC', 'CURY']

In [60]:
cur_discl

,Position Holder,Name of Share Issuer,ISIN,Net Short Position (%),Position Date
0,BlackRock Investment Management (UK) Limited,abrdn plc,GB00BF8Q6K64,1.05,2022-12-16
1,Citadel Advisors Europe Limited,abrdn plc,GB00BF8Q6K64,0.90,2022-11-22
2,Citadel Advisors LLC,abrdn plc,GB00BF8Q6K64,1.00,2022-12-29
3,GLG Partners LP,abrdn plc,GB00BF8Q6K64,1.15,2022-12-16
4,Qube Research & Technologies Limited,abrdn plc,GB00BF8Q6K64,0.55,2022-12-16
...,...,...,...,...,...
295,GLG Partners LP,WOOD GROUP (JOHN) PLC,GB00B5N0P849,0.68,2022-11-11
296,Qube Research & Technologies Limited,WOOD GROUP (JOHN) PLC,GB00B5N0P849,0.81,2022-12-16
297,Marshall Wace LLP,WPP PLC,JE00B8KF9B49,0.79,2022-12-13
298,GLG Partners LP,XP Power Limited,SG9999003735,1.06,2022-11-10


In [ ]:
n_bdays_ago
rept_date

In [59]:
pd.Series(isin_ticker_map, name='ticker').rename_axis('isin').reset_index()

,isin,ticker
0,GB0032089863,NXT
1,GB00B132NW22,ASHM
2,GB0030927254,ASC
3,GB00B4Y7R145,CURY
4,GB0033195214,KGF
5,GB00B53P2009,JUP
6,GB00BYRJ5J14,PHP
7,GB00B019KW72,SBRY
8,GB00BJTNFH41,AO/
9,GB00BZ6STL67,MTRO


In [10]:
query_start = datetime.today().date() - timedelta(days=50)

mkt_data = {tkr: query_mkt_data(qry_tkr, query_start) for tkr, qry_tkr in ticker_map.items()}
quotes = {tkr: query_quotes(qry_tkr) for tkr, qry_tkr in ticker_map.items()}

In [11]:
SH_OUT_COL = 'shares_outstanding'

mkt_data_df = process_mkt_data(mkt_data)
quotes_df = pd.DataFrame(quotes)
sh_out = quotes_df.loc['sharesOutstanding'].rename(SH_OUT_COL)

missing_sh_out = list(sh_out[sh_out.isna()].index)

if missing_sh_out:
    logger.warning(f"No share outstanding data for tickers: {missing_sh_out}")

No share outstanding data for tickers: ['VUKE']


In [50]:
from short_tracker.utils import setup_logging, n_bdays_ago

In [53]:
sh_out_ = sh_out.rename_axis("ticker").rename('value').to_frame().reset_index()
sh_out_.loc[:, 'date'] = pd.to_datetime(n_bdays_ago(1))
sh_out_.loc[:, 'item'] = 'shout'

In [54]:
sh_out_

,ticker,value,date,item
0,VUKE,NaN,2022-12-30,shout
1,NXT,129263000,2022-12-30,shout
2,ASHM,712740992,2022-12-30,shout
3,ASC,100015000,2022-12-30,shout
4,CURY,1133490048,2022-12-30,shout
5,KGF,1939730048,2022-12-30,shout
6,JUP,546387008,2022-12-30,shout
7,PHP,1336489984,2022-12-30,shout
8,SBRY,2345060096,2022-12-30,shout
9,AO/,576899968,2022-12-30,shout


In [41]:
mkt_data_df.rename_axis(index=DATE_COL, columns='item').set_index("Ticker", append=True).stack().rename('value').reset_index()

,date,Ticker,item,value
0,2022-11-11,VUKE,Close,32.025002
1,2022-11-11,VUKE,Adj Close,32.025002
2,2022-11-11,VUKE,Volume,276196.000000
3,2022-11-14,VUKE,Close,32.305000
4,2022-11-14,VUKE,Adj Close,32.305000
...,...,...,...,...
3157,2022-12-29,VCP,Adj Close,494.000000
3158,2022-12-29,VCP,Volume,106818.000000
3159,2022-12-30,VCP,Close,481.000000
3160,2022-12-30,VCP,Adj Close,481.000000


datetime.date(2022, 12, 30)

In [15]:
mkt_data_df.assign(**{DATE_COL: rept_date})

,Close,Adj Close,Volume,Ticker,Position Date
2021-12-30,32.334999,32.334999,181089,VUKE,2022-12-30
2021-12-31,32.259998,32.259998,63784,VUKE,2022-12-30
2022-01-04,32.775002,32.775002,922919,VUKE,2022-12-30
2022-01-05,32.860001,32.860001,379070,VUKE,2022-12-30
2022-01-06,32.570000,32.570000,505078,VUKE,2022-12-30
...,...,...,...,...,...
2022-12-22,34.950001,34.950001,10006476,BOO,2022-12-30
2022-12-23,34.750000,34.750000,9713914,BOO,2022-12-30
2022-12-28,33.889999,33.889999,13647584,BOO,2022-12-30
2022-12-29,36.000000,36.000000,9983660,BOO,2022-12-30


In [13]:
cur_discl

,Position Holder,Name of Share Issuer,ISIN,Net Short Position (%),Position Date
0,BlackRock Investment Management (UK) Limited,abrdn plc,GB00BF8Q6K64,1.05,2022-12-16
1,Citadel Advisors Europe Limited,abrdn plc,GB00BF8Q6K64,0.90,2022-11-22
2,Citadel Advisors LLC,abrdn plc,GB00BF8Q6K64,1.00,2022-12-29
3,GLG Partners LP,abrdn plc,GB00BF8Q6K64,1.15,2022-12-16
4,Qube Research & Technologies Limited,abrdn plc,GB00BF8Q6K64,0.55,2022-12-16
...,...,...,...,...,...
295,GLG Partners LP,WOOD GROUP (JOHN) PLC,GB00B5N0P849,0.68,2022-11-11
296,Qube Research & Technologies Limited,WOOD GROUP (JOHN) PLC,GB00B5N0P849,0.81,2022-12-16
297,Marshall Wace LLP,WPP PLC,JE00B8KF9B49,0.79,2022-12-13
298,GLG Partners LP,XP Power Limited,SG9999003735,1.06,2022-11-10


In [ ]:
ALPHAVANTAGE_ENDPOINT = "https://www.alphavantage.co/query?"

import requests

from short_tracker.data import APIRateLimitException

# def query_mkt_data(ticker, api_key):
#     """Query Alphavantage for adjusted stock prices and """
#     def query_endpoint(fn):
#         av_args = {"function": fn, "symbol": ticker, "apikey": api_key}
#         resp = requests.get(ALPHAVANTAGE_ENDPOINT, params=av_args)
#         resp.raise_for_status()
#         data = resp.json()
        
#         if "Note" in data:
#             raise APIRateLimitException
#         elif "Error Message" in data:
#             raise ValueError(data["Error Message"])
#         return data
        
#     price = query_endpoint("TIME_SERIES_DAILY_ADJUSTED")
#     sec_info = query_endpoint("OVERVIEW")
#     return price, sec_info

# price, sec_info = query_mkt_data("RWI.L", ALPHAVANGATE_KEY)

In [18]:
# TODO: nice to have schemas defined like this but keeping things simple for now

from typing import List
from sqlalchemy import Column, Float, String, create_engine
# from sqlalchemy.dialects.postgresql import ARRAY
from sqlalchemy.ext.declarative import declarative_base
from pydantic import BaseModel, constr

engine = create_engine("sqlite:///:memory:")

Base = declarative_base()


class DisclData(Base):
    __tablename__ = 'discl_data'
    fund = Column(String(200), primary_key=True, nullable=False)
    isin = Column(String(20), primary_key=True, nullable=False)
    share_issuer = Column(String(200), primary_key=True, nullable=False)
    short_position = Float()
    
    class Config:
        orm_mode = True


co_orm = DisclData(fund='a', isin='b', share_issuer='c', short_position=1.1231)
print(co_orm)

In [21]:
md = Base.metadata
md.create_all(engine)

In [22]:
md

FacadeDict({'discl_data': Table('discl_data', MetaData(), Column('fund', String(length=200), table=<discl_data>, primary_key=True, nullable=False), Column('isin', String(length=20), table=<discl_data>, primary_key=True, nullable=False), Column('share_issuer', String(length=200), table=<discl_data>, primary_key=True, nullable=False), schema=None)})

In [23]:
cur_discl

NameError: name 'cur_discl' is not defined

In [20]:
pd.read_sql_query('select * from discl_data', con=engine)

,fund,isin,share_issuer


In [33]:
cur_discl.to_sql(name='discl_data', con=engine, if_exists='append')

OperationalError: (sqlite3.OperationalError) table discl_data has no column named index
[SQL: INSERT INTO discl_data ("index", "Position Holder", "Name of Share Issuer", "ISIN", "Net Short Position (%)", "Position Date") VALUES (?, ?, ?, ?, ?, ?)]
[parameters: ((0, 'BlackRock Investment Management (UK) Limited', 'abrdn plc', 'GB00BF8Q6K64', 1.05, '2022-12-16 00:00:00.000000'), (1, 'Citadel Advisors Europe Limited', 'abrdn plc', 'GB00BF8Q6K64', 0.9, '2022-11-22 00:00:00.000000'), (2, 'Citadel Advisors LLC', 'abrdn plc', 'GB00BF8Q6K64', 1.0, '2022-12-29 00:00:00.000000'), (3, 'GLG Partners LP', 'abrdn plc', 'GB00BF8Q6K64', 1.15, '2022-12-16 00:00:00.000000'), (4, 'Qube Research & Technologies Limited', 'abrdn plc', 'GB00BF8Q6K64', 0.55, '2022-12-16 00:00:00.000000'), (5, 'Bronte Capital Management Pty Ltd', 'AFC ENERGY PLC', 'GB00B18S7B29', 0.5, '2022-03-31 00:00:00.000000'), (6, 'AKO Capital LLP', 'Alphawave IP Group PLC', 'GB00BNDRMJ14', 0.66, '2022-10-11 00:00:00.000000'), (7, 'KUVARI

In [34]:
cur_discl

,Position Holder,Name of Share Issuer,ISIN,Net Short Position (%),Position Date
0,BlackRock Investment Management (UK) Limited,abrdn plc,GB00BF8Q6K64,1.05,2022-12-16
1,Citadel Advisors Europe Limited,abrdn plc,GB00BF8Q6K64,0.90,2022-11-22
2,Citadel Advisors LLC,abrdn plc,GB00BF8Q6K64,1.00,2022-12-29
3,GLG Partners LP,abrdn plc,GB00BF8Q6K64,1.15,2022-12-16
4,Qube Research & Technologies Limited,abrdn plc,GB00BF8Q6K64,0.55,2022-12-16
...,...,...,...,...,...
295,GLG Partners LP,WOOD GROUP (JOHN) PLC,GB00B5N0P849,0.68,2022-11-11
296,Qube Research & Technologies Limited,WOOD GROUP (JOHN) PLC,GB00B5N0P849,0.81,2022-12-16
297,Marshall Wace LLP,WPP PLC,JE00B8KF9B49,0.79,2022-12-13
298,GLG Partners LP,XP Power Limited,SG9999003735,1.06,2022-11-10


In [1]:
from short_tracker.data_main import main as data_main
data_main()

No share outstanding data for tickers: ['VUKE']
/home/nikul/Documents/Programming/short_tracker/src/short_tracker/data_main.py:130: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  mkt_data.loc[:, DATE_COL] = pd.to_datetime(mkt_data[DATE_COL])
/home/nikul/Documents/Programming/short_tracker/src/short_tracker/data_main.py:130: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  mkt_data.loc[:, DATE_COL] = pd.to_datetime(mkt_data[DATE_COL])
/home/nikul/Documents/Programming/short_tracker/src/short_tracker/data_main.py:130: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to se

In [1]:
from short_tracker.metrics import main as metrics_main
metrics_main()

In [3]:
%load_ext autoreload
%autoreload 2


from short_tracker.config import (
    CONN_STR,
    MAX_DATA_AGE,
    TOP_N_SHORTS,
    UK_MKT_TICKER,
    ADV_CALC_LOOKBACK,
)

from short_tracker.data import FUND_COL, ADJ_CLOSE_COL, SH_OUT_COL, VOLUME_COL, TICKER_COL, DATE_COL, ISIN_COL, ITEM_COL, VALUE_COL, RET_COL, REL_RET_COL, REL_PNL_COL, PNL_COL, BM_RET_COL, EXPO_COL, AMOUNT_COL, SHARE_ISSUER_COL, SHORT_POS_COL, DTC_COL
from short_tracker.processing import reindex_rename_mkt_data, calc_reindex_dates, calc_returns, prepare_mkt_data, prepare_discl_data, subset_top_shorts
from short_tracker.metrics import query_all_db_data, summarise_short_discl, augment_discl_metrics, style_metrics_df
from short_tracker.utils import n_bdays_ago
    
discl_data, mkt_data, isin_ticker_map = query_all_db_data()

sec_short_metrics, fund_short_metrics = summarise_short_discl(discl_data, mkt_data, isin_ticker_map)

In [5]:
report_date = discl_data[DATE_COL].max()
df = fund_short_metrics

st = (
    style_metrics_df(df, report_date)
    # .set_table_styles([{"selector": "tr:hover", "props": "background-color: yellow"}])
)

with open('/home/nikul/Documents/tmp/index.html', 'w') as f:
    f.write(st.to_html())

st

Ticker,Name of Share Issuer,Position Holder,Net Short Position (%),Return,PnL,Days to Cover,Exposure,Exposure Change,Lookback Date
MTRO,METRO BANK PLC,ODEY ASSET MANAGEMENT LLP,4.1%,-3.0%,£267k,7.6,-£8.8M,£0k,22-12-30
FEVR,FEVERTREE DRINKS PLC,BlackRock Investment Management (UK) Limited,4.0%,-2.6%,"£1,249k",14.4,-£49.0M,£0k,22-12-30
ASC,ASOS PLC,Marshall Wace LLP,3.0%,-2.3%,£362k,3.9,-£15.8M,£0k,22-12-30
SBRY,SAINSBURY (J) PLC,BlackRock Investment Management (UK) Limited,2.6%,-0.6%,£840k,9.8,-£131.5M,£0k,22-12-30
VCP,VICTORIA PLC,"Coltrane Asset Management, L.P.",2.4%,-2.6%,£360k,7.9,-£13.7M,£0k,22-12-30
WINE,NAKED WINES PLC,Ennismore Fund Management Limited,2.4%,-1.7%,£39k,2.3,-£2.3M,£0k,22-12-30
EMIS,EMIS GROUP PLC,Beryl Capital Mangement LLC,2.3%,0.1%,-£29k,4.4,-£27.2M,£0k,22-12-30
ITM,ITM POWER PLC,Helikon Investments Limited,2.1%,-2.9%,£361k,4.4,-£12.4M,£0k,22-12-30
DTY,DIGNITY PLC,Ennismore Fund Management Limited,2.0%,-2.1%,£91k,5.7,-£4.3M,£0k,22-12-30
RWI,Renewi Plc,GLG Partners LP,2.0%,1.2%,-£110k,7.0,-£9.3M,£0k,22-12-30


In [101]:
style_metrics_df(fund_short_metrics.head(5)[DISPL_COLS])

In [449]:
from short_tracker.metrics import main as metrics_main
metrics_main()

TypeError: Object of type DataFrame is not JSON serializable

In [446]:
top_sec_aug

,date,Name of Share Issuer,ISIN,Net Short Position (%),Ticker,sharesOutstanding,Adj Close,Return,Volume,Benchmark Return,Relative Return,Amount,Exposure,PnL,Rel PnL,Days to Cover
0,2022-12-30,ASOS PLC,GB0030927254,0.0739,ASC,1.000150e+08,5.2250,-0.022967,7.701605e+05,-0.005182,-0.017784,7.391109e+06,-3.861854e+07,8.869330e+05,6.868041e+05,9.596842
1,2022-12-30,ITM POWER PLC,GB00B0130H42,0.0593,ITM,6.161580e+08,0.9438,-0.029032,3.016358e+06,-0.005182,-0.023849,3.653817e+07,-3.448472e+07,1.001145e+06,8.224384e+05,12.113342
2,2022-12-30,NAKED WINES PLC,GB00B021F836,0.0579,WINE,7.400410e+07,1.2900,-0.017054,7.568459e+05,-0.005182,-0.011872,4.284837e+06,-5.527440e+06,9.426629e+04,6.562200e+04,5.661439
3,2022-12-30,KINGFISHER PLC,GB0033195214,0.0575,KGF,1.939730e+09,2.3760,-0.006313,7.111877e+06,-0.005182,-0.001131,1.115345e+08,-2.650059e+08,1.673017e+06,2.997042e+05,15.682848
4,2022-12-30,BOOHOO GROUP PLC,JE00BG6L7297,0.0514,BOO,1.268190e+09,0.3600,-0.015833,1.200852e+07,-0.005182,-0.010651,6.518496e+07,-2.346659e+07,3.715541e+05,2.499456e+05,5.428227
5,2022-12-30,abrdn plc,GB00BF8Q6K64,0.0465,ABDN,2.008410e+09,1.8965,-0.002109,1.518932e+07,-0.005182,0.003073,9.339106e+07,-1.771161e+08,3.735586e+05,-5.442923e+05,6.148469
6,2022-12-30,HAMMERSON PLC,GB00BK7YQK64,0.0430,HMSO,4.994570e+09,0.2380,0.000000,9.366288e+06,-0.005182,0.005182,2.147665e+08,-5.111443e+07,-0.000000e+00,-2.648851e+05,22.929738
7,2022-12-30,TRAVIS PERKINS PLC,GB00BK9RKT01,0.0427,TPK,2.125090e+08,9.0740,-0.019837,6.751418e+05,-0.005182,-0.014655,9.074134e+06,-8.233869e+07,1.633344e+06,1.206649e+06,13.440338
8,2022-12-30,Currys plc,GB00B4Y7R145,0.0416,CURY,1.133490e+09,0.5460,-0.018315,4.210158e+06,-0.005182,-0.013133,4.715319e+07,-2.574564e+07,4.715319e+05,3.381129e+05,11.199862
9,2022-12-30,SAINSBURY (J) PLC,GB00B019KW72,0.0416,SBRY,2.345060e+09,2.1910,-0.006390,6.145962e+06,-0.005182,-0.001208,9.755450e+07,-2.137419e+08,1.365772e+06,2.581192e+05,15.872941


In [447]:
top_fund_aug

,ISIN,Position Holder,Name of Share Issuer,date,Net Short Position (%),Ticker,sharesOutstanding,Adj Close,Return,Volume,Benchmark Return,Relative Return,Amount,Exposure,PnL,Rel PnL,Days to Cover
0,GB00BZ6STL67,ODEY ASSET MANAGEMENT LLP,METRO BANK PLC,2022-12-30,0.0407,MTRO,1.725380e+08,1.2480,-0.030449,9.183970e+05,-0.005182,-0.025267,7.022297e+06,-8.763826e+06,2.668475e+05,2.214316e+05,7.646254
1,GB00BRJ9BJ26,BlackRock Investment Management (UK) Limited,FEVERTREE DRINKS PLC,2022-12-30,0.0397,FEVR,1.165640e+08,10.5800,-0.025520,3.215662e+05,-0.005182,-0.020338,4.627591e+06,-4.895991e+07,1.249450e+06,9.957296e+05,14.390786
2,GB0030927254,Marshall Wace LLP,ASOS PLC,2022-12-30,0.0302,ASC,1.000150e+08,5.2250,-0.022967,7.701605e+05,-0.005182,-0.017784,3.020453e+06,-1.578187e+07,3.624544e+05,2.806696e+05,3.921849
3,GB00B019KW72,BlackRock Investment Management (UK) Limited,SAINSBURY (J) PLC,2022-12-30,0.0256,SBRY,2.345060e+09,2.1910,-0.006390,6.145962e+06,-0.005182,-0.001208,6.003354e+07,-1.315335e+08,8.404750e+05,1.588426e+05,9.767964
4,GB00BZC0LC10,"Coltrane Asset Management, L.P.",VICTORIA PLC,2022-12-30,0.0241,VCP,1.150010e+08,4.9400,-0.026316,3.527453e+05,-0.005182,-0.021134,2.771524e+06,-1.369133e+07,3.602981e+05,2.893470e+05,7.857013
5,GB00B021F836,Ennismore Fund Management Limited,NAKED WINES PLC,2022-12-30,0.0240,WINE,7.400410e+07,1.2900,-0.017054,7.568459e+05,-0.005182,-0.011872,1.776098e+06,-2.291167e+06,3.907411e+04,2.720083e+04,2.346711
6,GB00B61D1Y04,Beryl Capital Mangement LLC,EMIS GROUP PLC,2022-12-30,0.0230,EMIS,6.331140e+07,18.7000,0.001070,3.331406e+05,-0.005182,0.006252,1.456162e+06,-2.723023e+07,-2.912324e+04,-1.702357e+05,4.371013
7,GB00B0130H42,Helikon Investments Limited,ITM POWER PLC,2022-12-30,0.0214,ITM,6.161580e+08,0.9438,-0.029032,3.016358e+06,-0.005182,-0.023849,1.318578e+07,-1.244474e+07,3.612901e+05,2.967990e+05,4.371425
8,GB00BRB37M78,Ennismore Fund Management Limited,DIGNITY PLC,2022-12-30,0.0201,DTY,5.004340e+07,4.2400,-0.021226,1.758369e+05,-0.005182,-0.016044,1.005872e+06,-4.264899e+06,9.052851e+04,6.842696e+04,5.720485
9,GB00BNR4T868,GLG Partners LP,Renewi Plc,2022-12-30,0.0196,RWI,8.022000e+07,5.9100,0.011844,2.245897e+05,-0.005182,0.017027,1.572312e+06,-9.292364e+06,-1.100618e+05,-1.582167e+05,7.000819


In [439]:
discl_data_aug[discl_data_aug[TICKER_COL]=='ASC']

,ISIN,Position Holder,Name of Share Issuer,date,Net Short Position (%),Ticker,sharesOutstanding,Adj Close,Return,Volume,Benchmark Return,Relative Return,Amount,Exposure,PnL,Rel PnL,Days to Cover
16,GB0030927254,AHL Partners LLP,ASOS PLC,2022-12-30,0.0090,ASC,100015000.0,5.225,-0.022967,770160.454545,-0.005182,-0.017784,900135.0,-4.703205e+06,108016.20,83643.261857,1.168763
17,GB0030927254,Citadel Advisors LLC,ASOS PLC,2022-12-30,0.0081,ASC,100015000.0,5.225,-0.022967,770160.454545,-0.005182,-0.017784,810121.5,-4.232885e+06,97214.58,75278.935671,1.051887
18,GB0030927254,GAM International Management Limited,ASOS PLC,2022-12-30,0.0178,ASC,100015000.0,5.225,-0.022967,770160.454545,-0.005182,-0.017784,1780267.0,-9.301895e+06,213632.04,165427.784561,2.311553
19,GB0030927254,Marshall Wace LLP,ASOS PLC,2022-12-30,0.0302,ASC,100015000.0,5.225,-0.022967,770160.454545,-0.005182,-0.017784,3020453.0,-1.578187e+07,362454.36,280669.612008,3.921849
20,GB0030927254,Squarepoint Ops LLC,ASOS PLC,2022-12-30,0.0088,ASC,100015000.0,5.225,-0.022967,770160.454545,-0.005182,-0.017784,880132.0,-4.598690e+06,105615.84,81784.522704,1.142790
